In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray  # activate the rio accessor

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import locale

locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import matplotlib.pyplot as plt
import matplotlib as mpl
# mpl.use('TkAgg')
import hvplot.xarray

import holoviews as hv
import holoviews.operation.datashader as hd

hv.extension('bokeh')
import colorcet as cc
import panel as pn
import panel.widgets as pnw
import ipywidgets as widgets

import pyproj as ppj
import rasterio
from affine import Affine
from shapely.geometry import box, Point, mapping
from shapely.ops import transform
import utils

u = utils
opj = os.path.join

# ## Set Dask local cluster

# In[2]:


import dask.distributed

cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
client

In [ ]:
# In[3]:


satdir = '/sat_data/satellite/sentinel2/L2A/GRS/31TGM'

image = 'S2A_MSIl2grs_20210906T103021_N0301_R108_T31TGM_20210906T141939_cc004_v14.nc'
file = glob.glob(opj(satdir, image))[0]


# In[4]:


def assign_coords(product, res=None):
    if res == None:
        res = float(product.metadata.attrs['Processing_Graph:node_1:parameters:resolution'])
    i2m = np.array((product.crs.i2m.split(','))).astype(float)
    geotransform = (i2m[4], i2m[0], i2m[1], i2m[5], i2m[2], i2m[3])
    fwd = Affine.from_gdal(*geotransform)
    x0, y0 = fwd * (0, 0)
    x1, y1 = fwd * (product.x.values[-1] + 1, product.y.values[-1] + 1)

    product['x'] = np.arange(x0 + res / 2, x1 - 1, res)
    product['y'] = np.arange(y0 - res / 2, y1 + 1, -res)
    return product


class masking():
    def __init__(self, product ,flag_ID='flags', names_='flag_meanings',
                  description_='flag_descriptions',
                  mask_binary='flag_masks'):
        self.product = product

        self.flag_ID=flag_ID
        self.names_=names_
        self.description_=description_
        self.mask_binary=mask_binary
        self.get_flags()

    def get_flags(self, ):

        pflags = self.product[self.flag_ID]
        names = []
        for flag_name in pflags.attrs[self.names_].split(' '):
            names.append(flag_name)

        # construct dataframe:
        dflags = pd.DataFrame({'name': names})
        dflags['description'] = pflags.attrs[self.description_].split('\t')
        dflags['value'] = pflags.attrs[self.mask_binary]
        dflags.sort_values('value',inplace=True)
        dflags['bit']=dflags.index
        self.dflags = dflags.set_index('name')
        self.pflags = pflags

    def print_info(self):
        return self.dflags 

    @staticmethod
    def bitmask(mask, bitval, value):
        """

        """

        if value:
            mask |= bitval
        else:
            mask &= (~bitval)
        return mask

    def compute_mask_value(self, **flags):

        mask = 0
        for flag_name, flag_ref in flags.items():
            bit, bit_val = self.dflags.loc[flag_name, ['bit','value']]
            print(flag_name, flag_ref, bit)
            mask = self.bitmask(mask, bit_val, True)
            value = self.bitmask(mask, bit_val, flag_ref)
            self.mask = mask
            self.value = value
        return mask, value

    def get_mask(self, **flags):
        """
        Returns boolean xarray computed from **flags
        Example:
        masking_ = masking(product)
        mask_ = masking_.make_mask(MG2_Water_Mask=False,negative=False,nodata=True)

        :param flags: list of boolean flags
        :return: boolean xarray.DataArray
        """

        mask, value = self.compute_mask_value(**flags)


        return self.product[self.flag_ID] & mask == value

In [ ]:
product = xr.open_dataset(file, chunks={'x': 512, 'y': 512}, decode_coords='all')
crs = ppj.CRS.from_wkt(product.crs.wkt)
# product = product.metpy.assign_crs(crs.to_cf()).metpy.assign_y_x()
product.rio.write_crs(crs, inplace=True)
product = assign_coords(product)
product.rio.write_coordinate_system()


# ## Check metadata
def subset_xy(ds, minx, miny, maxx, maxy):
    return ds.sel(x=slice(minx, maxx), y=slice(maxy, miny))

# # subset image

# In[69]:


minx, miny, maxx, maxy = 6.17, 46.1, 7.1, 46.55
minx, miny, maxx, maxy = 74e4, 5.115e6, 81e4, 5.16e6
#minx, miny, maxx, maxy = 75e4, 5.14e6, 76e4, 5.15e6

bbox = box(minx, miny, maxx, maxy)
p_ = subset_xy(product, minx, miny, maxx, maxy)
p_

## Check flags witihin the image

In [ ]:
masking_ = masking(p_)
masking_.print_info()

In [ ]:
plt.figure()
p_.Rrs_B3.plot(vmin=0,vmax=0.02,cmap=plt.cm.Spectral_r)
p_
plt.figure()
p_.flags.plot( norm=mpl.colors.LogNorm(vmin=0.1,vmax=2**12))


mask_ = masking_.get_mask(negative=True)
plt.figure()
mask_.plot()

# mask product
p_masked = p_.where(mask_)
plt.figure()
p_masked.Rrs_B3.plot(vmin=0,vmax=0.02,cmap=plt.cm.Spectral_r)